In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

Reading the dataset

In [52]:
data = pd.read_csv("train.csv")
data_test = pd.read_csv("test.csv")
data_test_out = pd.read_csv("gender_submission.csv")

Pre-processing of data: Removing columns which have no impact on deciding the survival of passengers.  

In [53]:
data.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
data_test.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)

Dividing the dataset into input and target

In [54]:
target=data.Survived
inputs=data.drop('Survived', axis='columns')
target_test=data_test_out
inputs_test=data_test

Changing the values of sex columns into numbers so that we can use them to train our model.

In [55]:
dummies=pd.get_dummies(inputs.Sex)
dummies_test=pd.get_dummies(inputs_test.Sex)

Merging the male female column into the original dataset.

In [56]:
inputs=pd.concat([inputs,dummies],axis='columns')
inputs_test=pd.concat([inputs_test,dummies_test],axis='columns')

Deleting Sex column as no longer required.

In [57]:
inputs.drop('Sex', axis='columns',inplace=True)
inputs_test.drop('Sex', axis='columns',inplace=True)
inputs_test.head(3)

,Pclass,Age,Fare,female,male
0,3,34.5,7.8292,0,1
1,3,47.0,7.0000,1,0
2,2,62.0,9.6875,0,1


Checking if any column has NA values

In [58]:
inputs.columns[inputs.isna().any()]
inputs.Age = inputs.Age.fillna(inputs.Age.mean())

Fill the NA value by taking avg of all Age's numeric value.

In [59]:
inputs_test.columns[inputs_test.isna().any()]
inputs_test.Age = inputs_test.Age.fillna(inputs_test.Age.mean())

Taking variable as k

In [60]:
k = 5

This method gives euclidien distance between any two data points

In [61]:
def euclidien_dist(x1, x2):
    count = 0
    for i in range(len(x1)):
        a=(x1[i]-x2[i])**2
        count = count + a
    return count ** 0.5

This method compares actual and predicted output

In [62]:
def compare(y_pred, y_test):
    count = 0
    for i in y_test.index:
        if y_pred[i] == y_test.loc[i]:
            count = count +1
    return count*100/len(y_pred)

This method trains the model with K nearest neighbours algorithm and renders accuracy by comparing predicted output with actual output

In [63]:
def knn(x_train, y_train, x_test, y_test, k):
    mini = 0
    count = 0
    c = []
    for index in x_test.index:
        dist_list = []
        for index1 in x_train.index:
            dist = euclidien_dist(x_test.loc[index], x_train.loc[index1])
            dist_list.append(dist)
        indices = np.argsort(dist_list)[:k]
        a = []
        for i in indices:
            a.append(y_train.loc[i])
        most_common = Counter(a).most_common(1)        
        c.append(most_common[0][0])
    return compare(c, y_test.Survived)
    

In [64]:
accuracy = knn(inputs, target, inputs_test,  target_test, k)
print("accuracy : ", accuracy, "%")

accuracy :  72.72727272727273 %
